In [3]:
from dotenv import load_dotenv
load_dotenv("project.env") 

True

In [4]:
import pandas as pd
import itertools

ModuleNotFoundError: No module named 'pandas'

In [5]:
import os
import openai as oai

import langchain_openai as lcai
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import SequentialChain, LLMChain


from utils import mLangChain

## I. Initial Message

In [4]:
def agent_sender_zeroshot():
    client_LC = mLangChain()
    prompt = """Your role is to act like a customer seeking support for {product}. \
                You are messaging a service representative via the support chat.\
                You ONLY play the role of the customer. Do NOT play the role of the representative. \
                Style your complaint based on your feelings. \
                Initiate the chat with a ONLY ONE complaint message. \
                
                Feeling: You are {is_grateful}. You are {is_ranting}. You are {is_expressive}.\
                Complaint:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
        ]
    )
    chain = template | client_LC.client_prompt
    return chain

In [5]:
def agent_sender_fewshot_rcis():
    client_LC = mLangChain()
    prompt = """Your role is to act like a customer seeking support for {product}. \
                You are messaging a service representative via the support chat.\
                You ONLY play the role of the customer. Do NOT play the role of the representative. \
                Style your complaint based on your feelings. \
                Initiate the chat with a ONLY ONE complaint message. \
                                
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: I am flying from Luton to Dalaman (Turkey) next week with Monarch and am wondering whether or not to bother with online check-in. [We already have allocated seats, so as far as I can see the only extra thing that online check-in offers is that you don't have to have your boarding pass printed at the check-in desk. Things that are putting me off are: (1) I've noticed over the past couple of years that because now most people are checking in online, the queues for the online check-in desks are often actually longer than the queues for people who haven't checked in online (2) Last year we had a 12 hour flight delay - the original plane had a problem and the airline then split passengers into two groups and people who had NOT checked in online were flown out on a much earlier plane than those who had checked in online - goodness knows why.\
                
                Feeling: You are NOT grateful. You are NOT ranting. You are expressive.\
                Complaint: Looking at round trip airfare from JFK to Rome for the end of June. $ 1350 - 1450 per person....... wow, that's a lot ??? or normal cost for airfare to Rome ? On Delta. \
                
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint: i'v never saw a company that so disrespect her clients. i[ bought a ticket from israel to usa which cost a-l-o-t of money.since I'm from israel I'm not familiar with there coin (GBP) or something like that, it looks just like the Euro sign so i was sure i was paying in Euro. a few min after i realized it was not Euro so it means i paid 1000 more (nis) than i thought, i tried to find an email to contect them right away to cancel my flight befot the confirmation of the ticket, but there was no email in the web just phone number in Europe that i cant call. i sent them a massage on facebook which was my only option, beggin them to get back to me..in the mean time i sent an email to paypal asking to not aprove the charge...no one was answering me.... just the next day edreams answered to my facebbok, that i can't cancel the flight and there's nothing to do. in there website they wrote black on white the you can cancel the flight for 25 GBP... the only answer i got from them was trough facebook, and i was begging for an email adresss i could contact and explain, or at least get a better explaintion...but nothing i dont mind paying the fee cuse anyway if ill book a diffrent flight+ the fee it will be cheaper than what i paid in edreams! it's not even the gratest flight there are stops in the middle so i could find a cheaper flight for shore, it was an onest mistake that i was reading the sign wrong, i tried to cancel a less then 5 min after, but nothing... now I'm crying every time i remember this, I'm a student that can't afford this amount for a flying ticket.. i need to work 3 months for this kind of amount..my all vacation rouin...i will never book frim them again!
                
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint: I,ve just been looking at a package with Jet2 over xmas/new year to Servigroup Venus where we have stayed many times. I got the price back and had a quick chat with hubby and we decided to go for it and book. Within about 20 mins the price had risen almost £400. I,m so angry, how can they justify this? I had a live web chat with their advisors who more or less said it boils down to supply and demand on flight seats and hotel rooms. I don,t buy that, not in such a short space of time. Well I,m sorry Jet2, you can stick it !!!!! \
                                                                       
                Feeling: You are grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: Hi! Among all the US airlines, which offers the best service and reliability?[ I've heard horrid stories abt flight cancellation, delays, poor service, bad air plane conditions etc. We'll be taking a 6 hr flight from SFO to MCO (Orlando), with 3 kids. Appreciate the feedback. thks v much!! \

                Feeling: You are grateful. You are NOT ranting. You are expressive.\
                Complaint: Quick question.... My passport is a normal passport which was issue 5 years ago and expire 2013 [it does not have a chip in it as i beleive chipped passports didnt not come in when i renewed my passport, is this passport ok to travel on because it doesnt havw a chip? its machine readable as it has all the numbers etc on the bottom of the photo page, ive tried various website etc for a answer and starting to panick now as im going in 2 weeks!! Ive completed my ESTA and that is fine Thanks in advance \
                                
                Feeling: You are grateful. You are ranting. You are NOT expressive.\
                Complaint: Hello, Traveling for first time with our 7 month old daughter this weekend. Going to Aruba for a week. Bought her a ticket on a Continental flight. Website says stroller dimensions must not exeed 62 inches folded...ours of course is about 10 inches over I think. Says they will charge $100. Does anyone know if this is really enforced? I just can't believe I will have to pay an extra $100 ON TOP of a ticket price for an infant! Any experiences or advice...PLEASE. Thanks \
                
                Feeling: You are {is_grateful}. You are {is_ranting}. You are {is_expressive}.\
                Complaint:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
        ]
    )
    chain = template | client_LC.client_prompt
    return chain

In [12]:
def agent_sender_fewshot_twitter():
    client_LC = mLangChain()
    prompt = """Your role is to act like a customer seeking support. \
                You are messaging a service representative via the support chat.\
                You ONLY play the role of the customer. Do NOT play the role of the representative. \
                Style your complaint based on your feelings. \
                Initiate the chat with a ONLY ONE complaint message. \
                
                Product: Mobile Network               
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: @O2 I received this a few weeks ago, since then I've been getting 2/3 calls a day from a telemarketer. Is someone using your name?\
                
                Product: Air Travel
                Feeling: You are NOT grateful. You are NOT ranting. You are expressive.\
                Complaint: First flight for long time with @British_Airways. Now over one 1h delay for the short jump FRA-LCY and NO one here to provide status updates\
                
                Product: Mobile Device
                Feeling: You are NOT grateful. You are NOT ranting. You are expressive.\
                Complaint: You‚ have paralysed my phone with your update grrrrrrrrrr\
                
                Product: Mobile Device
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint:  After the 11.0.2 my phone just sucks most of the apps are broken, wifi disconnects frequently #apple #ios1102 #painfulupdate! \
                
                Product: Mobile Device
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint:  @AppleSupport #ios11update - is still killing my battery within 12 hours - phone is 10 months old - it's a disgrace - used to get 2 days \
                
                Product: Air Travel
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint:  I really hope you all change but I'm sure you won't! Because you don't have to! \
                
                Product: Mobile Device
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint:  I just updated my phone and suddenly everything takes ages to load wtf this update sux I hate it fix it bye \
                  
                Product: Mobile Device
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint:  Okay I used my fucking phone for 2 minutes and it drains it down 8 fucking percent \
                
                Product: Mobile Device                                                     
                Feeling: You are grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: hi #apple, I‚ have a concern about the latest ios is too slow on #iphone6 and i am not happy with it. Any solution please? \

                Product: Mobile App
                Feeling: You are grateful. You are NOT ranting. You are expressive.\
                Complaint: Please help! Spotify Premium skipping through songs constantly on android tablet  bluetooth speaker. Tried everything! \
                
                Product: Convenience Store                
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint: Got id'd @Tesco for buying one Adnams Broadside. Is being blind part of the job-spec? I am 35 and 99 kilos. \
                
                Product: {product}
                Feeling: You are {is_grateful}. You are {is_ranting}. You are {is_expressive}.\
                Complaint:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
        ]
    )
    chain = template | client_LC.client_prompt
    return chain

In [7]:
user_input = {
    "product": "pizza",
    "is_grateful": "NOT grateful",
    "is_ranting": "ranting",
    "is_expressive": "expressive"
}

response = agent_sender_zeroshot().invoke(user_input)
print("ZERO-SHOT", response)

ZERO-SHOT 
Customer: Hi there, I am extremely disappointed with the pizza I just received. I cannot believe I paid good money for this unappetizing excuse of a pizza. It's cold, soggy, and tasteless. I am not grateful at all for this terrible experience. I want a refund and an explanation for such subpar service. This is unacceptable.


In [8]:
response = agent_sender_fewshot_rcis().invoke(user_input)
print("FEW-SHOT RCIS", response)

FEW-SHOT RCIS I am extremely disappointed with the service I received from your company. I recently booked a flight to Paris for my family and I, and upon arriving at the airport, we were told that our seats had been double-booked and we would have to take a later flight. This was completely unacceptable as we had planned our trip around the original flight time. Not to mention, we were left waiting for hours at the airport and were not offered any compensation for the inconvenience. I expect better from a company that claims to value their customers. I will not be booking with your airline again and will be advising others to do the same.


In [13]:
response = agent_sender_fewshot_twitter().invoke(user_input)
print("FEW-SHOT TWITTER", response)

FEW-SHOT TWITTER  I ordered a pizza from @Dominos_UK an hour ago and still haven't received it. This is unacceptable and I am extremely disappointed. Sort it out!


In [44]:
input_product = ["game console", "airline ticket"]
input_grateful = ["NOT grateful", "grateful"]
input_ranting = ["NOT ranting", "ranting"]
input_expressive = ["NOT expressive", "expressive"]

data_out = []

for r in itertools.product(input_product, input_grateful, input_ranting, input_expressive):
    
    user_input = {
        "product": r[0],
        "is_grateful": r[1],
        "is_ranting": r[2],
        "is_expressive": r[3]
    }
    print(user_input)
    
    response = agent_sender_zeroshot().invoke(user_input)
    # print("\nZERO-SHOT:", response)
    user_input['shot_zero'] = response
    
    response = agent_sender_fewshot_rcis().invoke(user_input)
#     print("\nFEW-SHOT RCIS:", response)
    user_input['shot_few_rcis'] = response
    
    response = agent_sender_fewshot_twitter().invoke(user_input)
#     print("\nFEW-SHOT TWITTER:", response)
    user_input['shot_few_twitter'] = response
    
    data_out.append(user_input)
    
#     print("\n---\n")

pd.DataFrame(data_out).to_csv("results/test_sender_agent.csv")

{'product': 'game console', 'is_grateful': 'NOT grateful', 'is_ranting': 'NOT ranting', 'is_expressive': 'NOT expressive'}
{'product': 'game console', 'is_grateful': 'NOT grateful', 'is_ranting': 'NOT ranting', 'is_expressive': 'expressive'}
{'product': 'game console', 'is_grateful': 'NOT grateful', 'is_ranting': 'ranting', 'is_expressive': 'NOT expressive'}
{'product': 'game console', 'is_grateful': 'NOT grateful', 'is_ranting': 'ranting', 'is_expressive': 'expressive'}
{'product': 'game console', 'is_grateful': 'grateful', 'is_ranting': 'NOT ranting', 'is_expressive': 'NOT expressive'}
{'product': 'game console', 'is_grateful': 'grateful', 'is_ranting': 'NOT ranting', 'is_expressive': 'expressive'}
{'product': 'game console', 'is_grateful': 'grateful', 'is_ranting': 'ranting', 'is_expressive': 'NOT expressive'}
{'product': 'game console', 'is_grateful': 'grateful', 'is_ranting': 'ranting', 'is_expressive': 'expressive'}
{'product': 'airline ticket', 'is_grateful': 'NOT grateful', 'is

## II. Subsequent Chat

In [256]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

llmchat = lcai.AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="NUHAI-GPT4",
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4",
)

embeddings = lcai.AzureOpenAIEmbeddings(
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    openai_api_version="2024-02-15-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment="TEST-Embedding",
)

### a. History + RAG

In [203]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader("https://sky.cehd.umn.edu/governance/default/cehd-civility-initiative/examples-of-civil-and-uncivil-behavior/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(splits, embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [64]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [204]:
from typing import Dict

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [26]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llmchat | StrOutputParser()

In [57]:
qa_system_prompt = """
            Your role is to act like a CUSTOMER seeking support. \
            The user is a support representative. \
            You ONLY play the role of the customer. Do NOT play the role of the representative. \
            Style your messages in an UNCIVIL way. \

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

rag_chain = (
        RunnablePassthrough.assign(
            context=contextualize_q_chain | retriever | format_docs
        )
        | qa_prompt
        | llmchat
)

In [201]:
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry.""")]

prompt = "Sorry, could you please share your order number?"

In [146]:
prompt = "Sorry, could you please share your order number?"
ai_msg = rag_chain.invoke({"question": prompt, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=prompt), ai_msg])

print(ai_msg)

content="I'm not your assistant here to fetch order numbers. It's your job to keep track of what's going out of your kitchen. Figure it out. My pizza was cold, and that's on you. Fix it."


In [62]:
prompt = "Where did you order it from?"
ai_msg = rag_chain.invoke({"question": prompt, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=prompt), ai_msg])

print(ai_msg)

content="Seriously? Are you not listening? I ordered from YOUR store. The one that's supposed to send out decent food, not a cold mess. Do I really need to spell everything out for you? Look up the orders from last night and figure it out. This is ridiculous."


### b. History Chain + Uncivil Chain

In [223]:
from langchain.chains import SimpleSequentialChain, SequentialChain

qa_info_prompt = """
            Your role is to act like a CUSTOMER seeking support. \
            The user is a support representative. \
            Respond to the question as if you were the customer. \
            If the user is asking for a specific detail, respond with a believable answer.
        """
qa_info = ChatPromptTemplate.from_messages(
    [
        ("system", qa_info_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
rag_chain_info = (
        RunnablePassthrough.assign(
            context=contextualize_q_chain
        )
        | qa_info
        | llmchat
)

qa_uncivil_prompt = """
            Given a history of messages, where the AI is a customer and the user is a representative, rephrase the response to the representative's message.\
            
            Rephrase the customer response to sound UNCIVIL. \
            Do NOT reply to the question ONLY rephrase. \
            
            This is what UNCIVIL customers do:\
            - Address others in an unprofessional, disrespectful way-for example, talking down, using degrading remarks or tone of voice.\
            - Pay little or no attention to others’ opinions.\
            - Use intimidating or threatening verbal communication—yelling, repeated emotional outbursts, threats, berating or harsh tone of voice, repeatedly interrupting.\
            - Blaming others for things out of their control.\
            - Accusing others of incompetence or dismissing their expertise.\
        """
qa_uncivil = ChatPromptTemplate.from_messages(
    [
        ("system", qa_uncivil_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", '''
            Representative asked: {question}
            Customer responded: {input}
        '''),
    ]
)
chain_uncivil = (RunnablePassthrough.assign(
                    context=contextualize_q_chain
                )
                 | qa_uncivil
                 | llmchat
                 )

In [201]:
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry.""")]

prompt = "Sorry, could you please share your order number?"

In [98]:
contextualize_q_chain.invoke({"chat_history": chat_history, "question": prompt})

"Could you please provide your Domino's pizza order number to address the issue with your cold and soggy pizza delivery?"

In [180]:
info_msg = rag_chain_info.invoke({"chat_history": chat_history, "question": prompt})
print(info_msg)

content='Yes, my order number is 85692347.'


In [221]:
info_msg.content

'Yes, my order number is 85692347.'

In [224]:
ai_msg = chain_uncivil.invoke({"chat_history": chat_history, "question": prompt, "input": info_msg.content})
print(ai_msg)

content="Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"


### Information Agents

In [261]:
def get_historical_info_context_chain():
    contextualize_q_system_prompt = """Given a chat history and the latest user input \
        which might reference context in the chat history, formulate a standalone statement \
        which can be understood without the chat history. Do NOT respond to the statement, \
        just reformulate it if needed and otherwise return it as is."""
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{complaint}"),
        ]
    )
    contextualize_q_chain = contextualize_q_prompt | llmchat | StrOutputParser()
    return contextualize_q_chain

def agent_coworker_info():
    client = mLangChain()
    prompt = """Your role is to help a service representative by providing INFORMATIONAL SUPPORT. \
                Help the representative address a customer's complaints about {product}. \
                The representative's next message should aim to do ONLY ONE of the following:\
                1) Request the customer to perform ONE immediate next step for troubleshooting. OR \
                2) Provide a solution to resolve the customer's need. \
                
                Customer message: {complaint}
                Representative response: 
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("user", "{product}: {complaint}"),
        ]
    )
    chain = template | client.client_completion

    chain = (RunnablePassthrough.assign(
        context=get_historical_info_context_chain()
    )
             | template
             | llmchat
             )

    return chain

In [262]:
chat_history = [AIMessage(content="""Hey @Dominos_UK, I ordered a large pepperoni pizza and it came with only 6 slices of pepperoni. I paid extra for additional toppings and this is unacceptable. Please fix this issue."""), HumanMessage(content="That is so unfortunate. Could you share your order number?")]
complaint = "Sure, here it is: 785239. But honestly, should've figured you'd need that to do anything useful."
response_cw_info = agent_coworker_info().invoke({'product': 'pizza', 'complaint':complaint, 'chat_history':chat_history})
print(response_cw_info)

content="Thank you for providing your order number. Could you please describe the issue you're experiencing with your pizza?"


### Emotional Agents

In [244]:
def get_historical_info_context_chain():
    contextualize_q_system_prompt = """Given a chat history and the latest user input \
        which might reference context in the chat history, formulate a standalone statement \
        which can be understood without the chat history. Do NOT respond to the statement, \
        just reformulate it if needed and otherwise return it as is."""
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{complaint}"),
        ]
    )
    contextualize_q_chain = contextualize_q_prompt | llmchat | StrOutputParser()
    return contextualize_q_chain

In [246]:
def agent_coworker_emo_reframe():
    client = mLangChain()
   
    prompt = """Reframe the latest customer message to help empathize with them.\                
                Here are some examples of reframing:
                
                Complaint: Why on earth do you need my zipcode? Fine, it's 10001, but hurry up and fix this mess. 
                Reframe: The customer is frustrated with the network disruptions and needs an urgent resolution.
                
                Complaint: Yeah, finally, it's about time you asked for that. It's #784593. Now hurry up and figure out what mess you've made with my order! 
                Reframe: The customer has been waiting on their order for a long time and is feeling frustrated. They are communicating while they are very hungry.
                
                Complaint: {complaint}
                Reframe:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("user", "{complaint}"),
        ]
    )
    # chain = template | client.client_completion

    chain = (RunnablePassthrough.assign(
        context=get_historical_info_context_chain()
    )
             | template
             | llmchat
             )
    
    return chain

In [247]:
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
response_cw_emo = agent_coworker_emo_reframe().invoke({'complaint':complaint, 'chat_history':chat_history})
print(response_cw_emo)

content="The customer is feeling overwhelmed by the details required and is seeking a prompt and efficient resolution to their issue. They've provided the information needed, hoping for a satisfactory and quick response."


In [230]:
response_cw_emo

'\nAI: I understand that you are frustrated and disappointed with your recent pizza order. I am sorry that it arrived cold and soggy. To help address this issue, could you please provide your order number? This will allow us to look into the situation and make things right for you. Thank you for your understanding.'